<a href="https://colab.research.google.com/github/ParushiSrivastava/disease-pridiction-w-drone/blob/main/disease_pridiction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"


In [3]:
!pip install -q tensorflow kagglehub

In [4]:
import tensorflow as tf

print("TensorFlow:", tf.__version__)
print("GPU:", tf.config.list_physical_devices('GPU'))


TensorFlow: 2.19.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
import random
import numpy as np

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Mixed precision (reduces memory usage)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

print("✅ GPU memory safe mode enabled")


✅ GPU memory safe mode enabled


In [6]:
import kagglehub

dataset_path = kagglehub.dataset_download(
    "abdallahalidev/plantvillage-dataset"
)

print("Dataset downloaded at:", dataset_path)


Using Colab cache for faster access to the 'plantvillage-dataset' dataset.
Dataset downloaded at: /kaggle/input/plantvillage-dataset


In [7]:
import os

color_dir = None
for root, dirs, files in os.walk(dataset_path):
    if "color" in dirs:
        color_dir = os.path.join(root, "color")
        break

if color_dir is None:
    raise FileNotFoundError("❌ color folder not found")

print("Using dataset path:", color_dir)


Using dataset path: /kaggle/input/plantvillage-dataset/plantvillage dataset/color


In [8]:
img_size = 224
batch_size = 16
epochs = 10


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = data_gen.flow_from_directory(
    color_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

validation_generator = data_gen.flow_from_directory(
    color_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)


Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.


In [10]:
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size


In [11]:
from tensorflow.keras import layers, models
tf.keras.backend.clear_session()

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu',
                  input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),

    layers.Dense(
        train_generator.num_classes,
        activation='softmax',
        dtype='float32'
    )
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    47,776,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 38)             │         9,766 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,805,158 (182.36 MB)

 Trainable params: 47,805,158 (182.36 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size,
    epochs=3
)


Epoch 1/3
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 323s 119ms/step - accuracy: 0.7587 - loss: 0.8092 - val_accuracy: 0.8675 - val_loss: 0.4355
Epoch 2/3
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 106s 39ms/step - accuracy: 0.9275 - loss: 0.2313 - val_accuracy: 0.8446 - val_loss: 0.5444
Epoch 3/3
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 108s 40ms/step - accuracy: 0.9648 - loss: 0.1111 - val_accuracy: 0.8575 - val_loss: 0.5766


In [15]:
val_loss, val_accuracy = model.evaluate(
    validation_generator,
    steps=validation_generator.samples // batch_size
)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


678/678 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.8565 - loss: 0.6041
Validation Accuracy: 85.75%


In [16]:
model.save("my_model.h5")


In [18]:
# Create train and validation generators
img_size = 224
batch_size = 16

train_generator = data_gen.flow_from_directory(
    color_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

validation_generator = data_gen.flow_from_directory(
    color_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

# ===================== COUNT PLANT SPECIES =====================

# Get all class labels
class_labels = list(train_generator.class_indices.keys())

# Extract plant names
plants = [label.split("___")[0] for label in class_labels]

# Unique plant species
unique_plants = set(plants)

print("\nTotal classes (plant + disease):", train_generator.num_classes)
print("Number of plant species:", len(unique_plants))
print("\nPlant species list:")
for plant in sorted(unique_plants):
    print("-", plant)

# ===================== COUNT DISEASE TYPES =====================

diseases = [label.split("___")[1] for label in class_labels]
unique_diseases = set(diseases)

print("\nNumber of disease categories:", len(unique_diseases))


Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.

Total classes (plant + disease): 38
Number of plant species: 14

Plant species list:
- Apple
- Blueberry
- Cherry_(including_sour)
- Corn_(maize)
- Grape
- Orange
- Peach
- Pepper,_bell
- Potato
- Raspberry
- Soybean
- Squash
- Strawberry
- Tomato

Number of disease categories: 21


In [20]:
from tensorflow.keras.models import load_model

model = load_model("my_model.h5")  # change name if different


In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Get class labels from generator
class_labels = list(train_generator.class_indices.keys())

def predict_plant_disease(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    class_index = np.argmax(predictions)
    class_label = class_labels[class_index]

    # Split plant and disease
    plant, disease = class_label.split("___")

    print("🌱 Plant:", plant)
    print("🦠 Disease:", disease)
    print("Confidence:", float(np.max(predictions)))


In [22]:
predict_plant_disease("/content/test_leaf.jpg")


FileNotFoundError: [Errno 2] No such file or directory: '/content/test_leaf.jpg'